In [ ]:
import sys
from PyQt5 import QtWidgets, QtCore, QtGui
import tkinter as tk
from PIL import ImageGrab
import numpy as np
import cv2
import io
import json
import ctypes
import PIL.Image
from io import BytesIO
import win32clipboard
import glob
from protobuf_to_dict import protobuf_to_dict
import os
from matplotlib import pyplot as plt
from IPython.display import clear_output, Image, display
from google.cloud.vision import types
from google.cloud import vision
import mss
from PyQt5.QtWidgets import QSystemTrayIcon
import pyperclip
import mss.tools
from google.protobuf.json_format import MessageToJson
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/poorv/Downloads/ocr_scripts/keys/direct-outlook-270501-c05c2d97d1c6.json"

In [ ]:
def send_image_to_clipboard(image):
    output = BytesIO()
    image.convert('RGB').save(output, 'BMP')
    data = output.getvalue()[14:]
    output.close()

    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(win32clipboard.CF_DIB, data)
    win32clipboard.CloseClipboard()

#image = Image.new("RGB", (200, 200), (255, 255, 0))
#send_to_clipboard(image)

In [ ]:
def send_text_clipboard(text):
    pyperclip.copy(text)
    spam = pyperclip.paste()

In [ ]:
# import googleapiclient.discovery
def dtlh(path, lh):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()
    if type(path) == str:
        with io.open(path, 'rb') as image_file:
            content = image_file.read()
            image = vision.types.Image(content=content)
    else:
        image = types.Image(content=cv2.imencode('.jpg', path)[1].tostring())
        
    response = client.text_detection(
    image=image,
    image_context={"language_hints": lh},  # Bengali
    )
    texts = response.text_annotations
#     print(texts)
#     print(type(response))
    
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

    return texts[0].description
#     serialized = MessageToJson(response)
#     with open('data.json', 'w') as f:
#         json.dump(response, f)
#     print(texts)
#     protobuf_to_dict(MyMessage().ParseFromString(response))
#     service = googleapiclient.discovery.build('vision', 'v1')
#     response = service_request.execute()
#     print(response['responses'])

    

#     response = MessageToJson(response, preserving_proto_field_name = True)
#     desired_res = response["label_annotation"]
#     return texts

# dtlh(r"C:\Users\poorv\Downloads\ocr_scripts\pngs\output-73-to-73.png", ["kn"])

In [ ]:
from PyQt5.QtGui import QIcon
class MyWidget(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        root = tk.Tk()
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        self.setGeometry(0, 0, screen_width, screen_height)
        self.setWindowTitle(' ')
        self.begin = QtCore.QPoint()
        self.end = QtCore.QPoint()
        self.setWindowOpacity(0.3)
        QtWidgets.QApplication.setOverrideCursor(
            QtGui.QCursor(QtCore.Qt.CrossCursor)
        )
        self.setWindowFlags(QtCore.Qt.FramelessWindowHint)
        QtWidgets.QShortcut(
            QtGui.QKeySequence("Escape"), self, activated=self.on_Escape
        )
        self.tray_icon = SystemTrayIcon(QtGui.QIcon(image), w)
#         self.trayIcon.setIcon(QIcon("icon.jpg"))
        print('Capture the screen...')
        self.show()

    def paintEvent(self, event):
        qp = QtGui.QPainter(self)
        qp.setPen(QtGui.QPen(QtGui.QColor('black'), 3))
        qp.setBrush(QtGui.QColor(128, 128, 255, 128))
        qp.drawRect(QtCore.QRect(self.begin, self.end))

    def mousePressEvent(self, event):
        self.begin = event.pos()
        self.end = self.begin
        self.update()
    
    @QtCore.pyqtSlot()
    def on_Escape(self):
        self.close()

    def mouseMoveEvent(self, event):
        self.end = event.pos()
        self.update()

    def mouseReleaseEvent(self, event):
        self.close()

        x1 = min(self.begin.x(), self.end.x())
        y1 = min(self.begin.y(), self.end.y())
        x2 = max(self.begin.x(), self.end.x())
        y2 = max(self.begin.y(), self.end.y())

#         img = ImageGrab.grab(bbox=(x1, y1, x2, y2))
#         img = cv2.UMat(img, cv2.COLOR_RGB2GRAY)
#         img.save('capture.png')
#         screen = np.asarray(sct.grab(monitor))
        h = abs(y2 - y1)
        w = abs(x2 - x1)
        with mss.mss() as sct:
            # The screen part to capture
            monitor = {"top": y1, "left":x1, "width":w , "height": h}
            
            output = "sct-{top}x{left}_{width}x{height}.png".format(**monitor)

            sct_img = sct.grab(monitor)
            img = PIL.Image.frombytes("RGB", sct_img.size, sct_img.bgra, "raw", "BGRX")

        img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
        im_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(im_gray,127,255,cv2.THRESH_BINARY_INV)
        ocr = dtlh(thresh, ["hi"])
#         send_text_clipboard(ocr)
        print(ocr)

#         cv2.imshow('Captured Image', img)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

In [ ]:
app = QtWidgets.QApplication(sys.argv)
window = MyWidget()
window.show()
app.aboutToQuit.connect(app.deleteLater)
sys.exit(app.exec_())

In [ ]:
%run -i 'python gcv2hocr/gcv2hocr.py test.jpg.json output.hocr'

In [ ]:
# %run "gcv2hocr/gcv2hocr.py" response > output.hocr

%run gcv2hocr/gcv2hocr.py response


In [ ]:
import pystray

icon = pystray.Icon('test name')

In [ ]:
from PIL import Image, ImageDraw

def create_image():
    # Generate an image and draw a pattern
    width = 25
    height = 25
    color1 = (255, 255, 0)
    color2 = (255, 0, 255)
    image = Image.new("RGB", (width, height), color1)
#     image = Image.new('RGB', (width, height), color1)
    dc = ImageDraw.Draw(image)
    dc.rectangle(
        (width // 2, 0, width, height // 2),
        fill=color2)
    dc.rectangle(
        (0, height // 2, width // 2, height),
        fill=color2)

    return image

icon.icon = create_image()